In [107]:
import torch
from torch import nn
import re
import random
import tqdm
import numpy as np
import time
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
X_1 = torch.randint(0,9,(5000,25))
X_2 = torch.randint(0,9,(5000,75))
X_3 = torch.randint(0,9,(5000,125))


In [3]:
def make_target(X_1):
  Y = X_1.clone()
  Y[:, 1:] = (Y[:, 1:] + Y[:,[0]])%10
  return Y

In [4]:
Y_1 = make_target(X_1)
Y_2 = make_target(X_2)
Y_3 = make_target(X_3)



In [96]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass):
        super().__init__()

        self.embedding = nn.Embedding(10, 128)
        self.hidden = rnnClass(128, 256, batch_first=True)
      
      
        self.output = nn.Linear(256, 10)

    def forward(self, X):
        out = self.embedding(X)
        o, state = self.hidden(out)
        o, state = self.hidden(out,state)
        predictions = self.output(o)
        return predictions

In [109]:
 def train_test(X,Y):

  X_train = X

  Y_train = Y

   
  results = {'LSTM':[],'GRU':[],'RNN':[]}
  for j,model_type in enumerate(['LSTM','GRU','RNN']):
    if model_type == 'LSTM':
      model = NeuralNetwork(nn.LSTM)
    elif model_type == 'RNN': 
      model = NeuralNetwork(nn.RNN)
    else:
      model = NeuralNetwork(nn.GRU)
      
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    for ep in range(100):
      start = time.time()
      train_loss = 0.
      train_passed = 0
  
      model.train()
     
    #  X_b, y_b = X_b.cuda(), y_b.cuda()
      optimizer.zero_grad()
      o = model(X_train)
      w =  torch.argmax(o[-1,:],keepdim=True)
      answers= o.view(-1,10)

      loss = criterion(answers, Y_train.flatten())
      train_loss += loss.item()

      loss.backward()
      optimizer.step()
      train_passed += 1
        
      model.eval()

      acc_sum, n = 0, 0
    

      
      y_pred = model(X_train)

      y_pred = y_pred.argmax(dim=2)
      
      acc_sum += (y_pred == Y_train).sum()
    #  print(acc_sum)
      n += Y.flatten().shape[0]
      results[model_type].append(acc_sum.item() / n)  
      if ep%2 == 0 :
        print(f'Epoch {ep} AUC = ',results[model_type][-1]*100,'%')
      
        #print("Epoch {ep}. Time: {time.time() - start}, AUC {results[model_type][-1]}".format(ep, time.time() - start,  results[model_type][-1]))
        
  return results


In [102]:
res_1 = train_test(X_1,Y_1)

Epoch 0 AUC =  11.484 %
Epoch 2 AUC =  13.1296 %
Epoch 4 AUC =  14.080799999999998 %
Epoch 6 AUC =  14.4512 %
Epoch 8 AUC =  14.6008 %
Epoch 10 AUC =  14.669599999999999 %
Epoch 12 AUC =  14.680000000000001 %
Epoch 14 AUC =  14.804 %
Epoch 16 AUC =  14.935200000000002 %
Epoch 18 AUC =  15.0256 %
Epoch 20 AUC =  15.1088 %
Epoch 22 AUC =  15.328 %
Epoch 24 AUC =  15.4504 %
Epoch 26 AUC =  15.634400000000001 %
Epoch 28 AUC =  15.9632 %
Epoch 30 AUC =  16.3408 %
Epoch 32 AUC =  16.9144 %
Epoch 34 AUC =  17.724 %
Epoch 36 AUC =  18.604799999999997 %
Epoch 38 AUC =  20.2624 %
Epoch 40 AUC =  22.66 %
Epoch 42 AUC =  23.2624 %
Epoch 44 AUC =  26.8984 %
Epoch 46 AUC =  31.868800000000004 %
Epoch 48 AUC =  35.7128 %
Epoch 50 AUC =  38.9568 %
Epoch 52 AUC =  43.7376 %
Epoch 54 AUC =  48.6856 %
Epoch 56 AUC =  52.480000000000004 %
Epoch 58 AUC =  59.628800000000005 %
Epoch 60 AUC =  61.74079999999999 %
Epoch 62 AUC =  64.9712 %
Epoch 64 AUC =  70.7416 %
Epoch 66 AUC =  74.77279999999999 %
Epoch 68

In [108]:
pd.DataFrame(res_1)

,LSTM,GRU,RNN
0,0.114840,0.121440,0.115016
1,0.123824,0.128648,0.136400
2,0.131296,0.133928,0.142816
3,0.135512,0.139192,0.145080
4,0.140808,0.142704,0.146192
...,...,...,...
95,0.964544,0.868240,0.217560
96,0.964528,0.890488,0.208312
97,0.964576,0.909856,0.197224
98,0.964560,0.920344,0.198200


In [110]:
res_2 = train_test(X_2,Y_2)

Epoch 0 AUC =  11.037333333333335 %
Epoch 2 AUC =  11.191733333333334 %
Epoch 4 AUC =  11.754133333333334 %
Epoch 6 AUC =  12.037066666666666 %
Epoch 8 AUC =  12.210933333333333 %
Epoch 10 AUC =  12.274933333333333 %
Epoch 12 AUC =  12.314666666666668 %
Epoch 14 AUC =  12.368 %
Epoch 16 AUC =  12.378133333333333 %
Epoch 18 AUC =  12.056533333333332 %
Epoch 20 AUC =  12.038133333333334 %
Epoch 22 AUC =  12.295466666666668 %
Epoch 24 AUC =  12.553066666666668 %
Epoch 26 AUC =  12.650133333333333 %
Epoch 28 AUC =  12.654399999999999 %
Epoch 30 AUC =  12.7392 %
Epoch 32 AUC =  12.783466666666667 %
Epoch 34 AUC =  12.784533333333334 %
Epoch 36 AUC =  12.865866666666667 %
Epoch 38 AUC =  12.934133333333333 %
Epoch 40 AUC =  13.005866666666666 %
Epoch 42 AUC =  13.0464 %
Epoch 44 AUC =  13.138666666666667 %
Epoch 46 AUC =  13.1984 %
Epoch 48 AUC =  13.357066666666666 %
Epoch 50 AUC =  13.455466666666666 %
Epoch 52 AUC =  13.5984 %
Epoch 54 AUC =  13.604266666666668 %
Epoch 56 AUC =  14.136533

In [111]:
pd.DataFrame(res_2)

,LSTM,GRU,RNN
0,0.110373,0.113720,0.111187
1,0.111261,0.114288,0.115565
2,0.111917,0.116840,0.117760
3,0.113760,0.118400,0.120600
4,0.117541,0.118483,0.122216
...,...,...,...
95,0.136443,0.142389,0.200992
96,0.137245,0.142755,0.202571
97,0.138053,0.143701,0.203965
98,0.139597,0.144325,0.206443


In [ ]:
res_3 = train_test(X_3,Y_3)

Epoch 0 AUC =  10.889759999999999 %
Epoch 2 AUC =  11.22672 %
Epoch 4 AUC =  11.466560000000001 %
Epoch 6 AUC =  11.56752 %
Epoch 8 AUC =  11.600000000000001 %
Epoch 10 AUC =  11.643040000000001 %
Epoch 12 AUC =  11.70848 %
Epoch 14 AUC =  11.70736 %
Epoch 16 AUC =  11.70464 %
Epoch 18 AUC =  11.50688 %
Epoch 20 AUC =  11.602079999999999 %
Epoch 22 AUC =  11.89408 %
Epoch 24 AUC =  11.98016 %
Epoch 26 AUC =  12.04832 %
Epoch 28 AUC =  12.13072 %
Epoch 30 AUC =  12.15856 %
Epoch 32 AUC =  12.11456 %
Epoch 34 AUC =  12.167200000000001 %
Epoch 36 AUC =  12.263359999999999 %
Epoch 38 AUC =  12.27456 %
Epoch 40 AUC =  12.36064 %
Epoch 42 AUC =  12.410400000000001 %
Epoch 44 AUC =  12.42848 %
Epoch 46 AUC =  12.42112 %
Epoch 48 AUC =  12.4704 %
Epoch 50 AUC =  12.538079999999999 %
Epoch 52 AUC =  12.60512 %
Epoch 54 AUC =  12.60752 %
Epoch 56 AUC =  12.68416 %
Epoch 58 AUC =  12.72016 %
Epoch 60 AUC =  12.766079999999999 %
Epoch 62 AUC =  12.8312 %
Epoch 64 AUC =  12.897120000000001 %
Epoch 

In [113]:
pd.DataFrame(res_3)

,LSTM,GRU,RNN
0,0.108898,0.111270,0.113424
1,0.110522,0.111986,0.113019
2,0.112267,0.113875,0.113030
3,0.113426,0.114837,0.115150
4,0.114666,0.115088,0.115933
...,...,...,...
95,0.162726,0.129589,0.130968
96,0.162387,0.129653,0.131134
97,0.162853,0.129792,0.131312
98,0.166181,0.129917,0.131483
